In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("./Cricket Dataset.csv")

C:\Users\debdi\AppData\Local\Temp\ipykernel_29928\3141108476.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Cricket Dataset.csv")


In [7]:
df.head(15)

,Balls,MatchNo,Innings,BattingTeam,City,Date,Match_Type,Toss_Decision,Toss_Winner,Venue,...,Temp,Score,Wickets,Wicket_ball,Ext_bl,fsdasfasd,Ball_b,Wicket_kind,Wicket_Fielders,Player_out
0,1,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,1,0,0,1,0,1,NaN,NaN,NaN
1,2,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,1,0,0,1,1,1,NaN,NaN,NaN
2,3,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,1,0,0,1,2,2,NaN,NaN,NaN
3,4,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,1,0,0,1,3,3,NaN,NaN,NaN
4,5,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,4,4,NaN,NaN,NaN
5,6,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,5,5,NaN,NaN,NaN
6,7,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,6,6,NaN,NaN,NaN
7,8,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,7,7,NaN,NaN,NaN
8,9,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,8,8,NaN,NaN,NaN
9,10,1,1,Australia,London,9/4/2009,ODI,field,England,Kennington Oval,...,1,5,0,0,1,9,9,NaN,NaN,NaN


In [6]:
selected_columns = df.columns[[1,2,13,22,24,25,26,29]]
selected_columns

Index(['MatchNo', 'Innings', 'Over', 'Runs_Total', 'Score', 'Wickets',
       'Wicket_ball', 'Ball_b'],
      dtype='object')

### Description of Selected Columns

- **MatchNo**: The unique number assigned to the match.  
- **Innings**: Indicates whether the data corresponds to the 1st or 2nd innings (values: 1 or 2).  
- **Over**: Represents the current over in the match, formatted as `x.y` (e.g., `0.1`, `0.5`, `3.6`), where `x` is the over count and `y` is the ball number within the over.  
- **Runs_Total**: The total runs scored on that particular ball, including extras.  
- **Score**: The cumulative score of the batting team at that point in the match.  
- **Wickets**: The total number of wickets lost by the batting team so far.  
- **Wicket_ball**: A binary field (`0` or `1`) indicating whether the given ball resulted in a wicket (`1` if a wicket fell, `0` otherwise).  
- **Ball_b**: The sequential ball number in the match (e.g., in an ODI, it will be at least 300 or more if there are extra deliveries due to no-balls or wides).  

In [14]:
data = df[selected_columns]
data = data[data["Innings"] == 1]

In [15]:
data.head()

,MatchNo,Innings,Over,Runs_Total,Score,Wickets,Wicket_ball,Ball_b
0,1,1,0.1,1,1,0,0,1
1,1,1,0.2,0,1,0,0,1
2,1,1,0.3,0,1,0,0,2
3,1,1,0.4,0,1,0,0,3
4,1,1,0.5,4,5,0,0,4


### Calculating R(b, w) which represents the average runs scored at a specific ball number b and a given number of wickets lost w.

In [23]:
sum_run = np.zeros((300, 11))
count = np.zeros((300, 11))
runbw = np.zeros((300, 11))

for i, row in data.iterrows():
    balls = int(row["Ball_b"]) 
    wickets = int(row["Wickets"])  
    runs = int(row["Runs_Total"])  

    if 1 <= balls <= 300 and 0 <= wickets <= 10:
        sum_run[balls - 1, wickets] += runs  
        count[balls - 1, wickets] += 1  

for i in range(0, 300):
    for j in range(0, 11):
        if count[i, j] > 0:
            runbw[i, j] = sum_run[i, j] / count[i, j]

### Calculating P(b, w) which represents the probability of a wicket falling at a specific ball b when w wickets have already fallen.

In [24]:
sum_prob_wicket = np.zeros((300, 11))
pbw = np.zeros((300, 11))

for i in range(len(data) - 1):
    balls = int(data.iloc[i]["Ball_b"])
    wickets = int(data.iloc[i]["Wickets"])
    wicket_ball = int(data.iloc[i]["Wicket_ball"])
    
    if 1 <= balls <= 300 and 0 <= wickets <= 10:
        sum_prob_wicket[balls - 1, wickets] += wicket_ball
        
for i in range(300):
    for j in range(11):
        if count[i, j] > 0:
            pbw[i, j] = sum_prob_wicket[i, j] / count[i, j]

### **Understanding the Formula for \( V(b, w) \)**  

The formula used in the dynamic programming approach to compute \( V(b, w) \) is:  

V(b,w) = runbw(b,w) + pbw(b,w) * V(b+1,w+1) + (1−pbw(b,w)) ⋅ V(b+1,w)

where:  

- **\( V(b, w) \)** → Expected total runs from ball \( b \) onwards when \( w \) wickets have fallen.  
- **\( {runbw}(b, w) \)** → The average number of runs scored on ball \( b \) when \( w \) wickets have fallen.  
- **\( {pbw}(b, w) \)** → The probability of a wicket falling on ball \( b \) when \( w \) wickets have fallen.  
- **\( V(b+1, w+1) \)** → The expected runs from ball \( b+1 \) onwards if a wicket falls.  
- **\( V(b+1, w) \)** → The expected runs from ball \( b+1 \) onwards if no wicket falls.  

### **Breakdown of the Formula**  

1. **Immediate Reward (\( {runbw}(b, w) \))**  
   - This is the average number of runs scored on ball \( b \) when \( w \) wickets have fallen.  
   - It represents the **immediate contribution to the score**.  

2. **Future Expectations (\( V(b+1, w+1) \) and \( V(b+1, w) \))**  
   - If a **wicket falls**, the match continues from \( (b+1, w+1) \) → weighted by \( {pbw}(b, w) \).  
   - If a **wicket does not fall**, the match continues from \( (b+1, w) \) → weighted by \( (1 - {pbw}(b, w)) \).  

### **Interpretation**  

- The formula balances **current run expectation** with **future possibilities** based on the probability of a wicket falling.  
- The backward computation ensures that **earlier balls** are aware of how the innings will play out in the future.  
- The first computed value \( V(1, 0) \) gives the **expected total runs at the start of the innings**.  

This approach is a form of **Markov Decision Process (MDP)**, where each ball’s outcome depends on its probability and leads to different future states.


In [28]:
V = np.zeros((300, 11))

for i in range(298, -1, -1): 
    for j in range(9, -1, -1):
        V[i, j] = runbw[i, j] + pbw[i, j] * V[i + 1, j + 1] + (1 - pbw[i, j]) * V[i + 1, j]

### Testing through Monte Carlo SImulation

In [39]:
N = 1000
simulated_scores = []

for _ in range(N):
    balls = 1
    wickets = 0
    total_runs = 0
    
    while balls <= 300 and wickets < 10:
        if np.random.rand() < pbw[balls - 1, wickets]:
            wickets += 1
        else:
            total_runs += np.random.poisson(runbw[balls - 1, wickets])

        balls += 1
    
    simulated_scores.append(total_runs)
    
mean_simulated_score = np.mean(simulated_scores)
std_simulated_score = np.std(simulated_scores)

print(f"Monte Carlo Mean Score: {mean_simulated_score}")
print(f"Monte Carlo Score Std Dev: {std_simulated_score}")
print(f"Predicted V(1,0): {V[0,0]}")

Monte Carlo Mean Score: 277.26
Monte Carlo Score Std Dev: 16.196555189298742
Predicted V(1,0): 277.9315138905608
